In [58]:
!pip install -r requirements.txt
!pip install jupyter-lsp
!pip install 'python-lsp-server[all]'
!pip install "pymongo[srv]"
!%config Completer.use_jedi = False
!ln -s / .lsp_symlink

/bin/bash: line 0: fg: no job control
ln: failed to create symbolic link '.lsp_symlink/': File exists


In [4]:
import torch
import numpy as np
import nltk
nltk.download("stopwords")
nltk.download("punkt")

from persistence import Connection as Database

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from typing import List

In [7]:
torch.cuda.is_available()

True

In [8]:
# load some data 
db: Database = Database()

Connected to DB


In [9]:
resumes = db.fetch_all_labeled_resumes()

In [10]:
from model.Job import Job

In [62]:
bag_of_words_DS = {
"machine learning": 0.5,
"r (programming language)": 0.5,
"data science": 0.5,
"python": 0.5,
"deep learning": 0.5,
"natural language processing (nlp)": 0.5,
"tableau": 0.5,
"sql": 0.5,
"predictive modeling": 0.5,
"statistics": 0.5,
"data analysis": 0.5,
"data visualization": 0.5,
"statistical modeling": 0.5,
"tensorflow": 0.5,
"predictive analytics": 0.5,
"artificial intelligence (ai)": 0.5,
"data mining": 0.5,
"scikit-learn": 0.5,
"data analytics": 0.5,
"apache": 0.5,
"spark": 0.5,
"big data": 0.5,
"hadoop": 0.5,
"algorithms": 0.5,
"git": 0.5,
"amazon web services (aws)": 0.5,
"big data analytics": 0.5,
"hive": 0.5,
"scala": 0.5,
"matlab": 0.5,
"pandas (software)": 0.5,
"numpy": 0.5,
"sas": 0.5,
"microsoft power bi": 0.5,
    "degree": 0.9,
    "data": 0.8
}


DS = Job("DS")
DS.set_keywords(bag_of_words_DS)


# Observations
- We observed that it/IT is a conflict with stop-word removal …
- Take the length of the experience into account 
- For example, statistical and statsitcs are close, maybe relevant to use embeddnings istead of intersection

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
  
stop_words = set(stopwords.words('english'))

def extract_keywords(text: str) -> List[str]: 
    words = set(tokenizer.tokenize(text.lower()))
    words = words.difference(stop_words)
    
    return words

In [ ]:
first = resumes[0]


for exp in first['experience']['value']:
    desc = exp['value']['description']
    print(desc)
    keywords = extract_keywords(desc)
    print(keywords)


set()
A>RACE is the Accenture Risk Analytics Center of Excellence. It is a center created within Accenture which deals with risk management projects with the objective to provide innovative IT solutions.
{'risk', 'objective', 'management', 'race', 'created', 'deals', 'projects', 'solutions', 'center', 'accenture', 'within', 'analytics', 'excellence', 'innovative', 'provide'}

set()
I was the tutor of the Master degree Direzione aziendale taught in the University of Bologna for one year. I managed the relations between professors and students, organized the schedules for the academic year and dealt with administrative issues related to the course's admission and graduation.
{'tutor', 'admission', 'taught', 'dealt', 'direzione', 'schedules', 'graduation', 'year', 'degree', 'administrative', 'professors', 'master', 'students', 'bologna', 'one', 'course', 'issues', 'managed', 'relations', 'academic', 'aziendale', 'university', 'related', 'organized'}
Mentee in the ICARO project. The proje

In [16]:
scores = list(map(score_experience, [exp['value'] for exp in first['experience']['value']]))

print(scores)

[0, 0, 0, 0.9, 0, 0]


In [17]:
print(first['education']['value'][0])

{'value': {'institution': 'Università di Bologna', 'title': 'Second cycle degree, Statistical science', 'location': 'Bologna', 'fromDate': '2017', 'toDate': '2019', 'description': ''}, 'label': 0}


In [18]:
def score_education_relevance(education) -> float: 
    title = education['title']
    keywords = extract_keywords(title)
    print(keywords)
    return DS.relevance(keywords)
    
    
edu_scores = list(map(score_education_relevance, [edu['value'] for edu in first['education']['value']]))
edu_scores

{'science', 'degree', 'cycle', 'statistical', 'second'}
{'student', 'erasmus', 'exchange'}
{'management', 'marketing', 'e', 'triennale', 'economia', 'aziendale', 'gestione', 'laurea', 'dell', 'della', 'scienze'}
set()


[0.9, 0, 0, 0]

In [19]:
from persistence.models import Resume

In [95]:
from copy import deepcopy
def score_resume(resume: Resume, job: Job) -> Resume:
    """
    Takes in a resume and returns the same resume with labels given by our
    scoring models .
    This can be used as input for shortening the resume
    """
    
    # Create a deep copy for the output to make sure
    # we dont modify the input data in any way 
    output = deepcopy(resume)
    
    # summary 
    summary_score = job.relevance(extract_keywords(resume['summary']['value']))
    output['summary']['label'] = summary_score
    
    
    #experience
    for exp in output['experience']['value']:
        exp_keyword = extract_keywords(exp['value']['description'])
        score = job.relevance(exp_keyword)
        exp['label'] = score
    
    
    # Education
    for edu in output['education']['value']:
        edu_keyword = extract_keywords(edu['value']['title'])
        score = job.relevance(edu_keyword)
        edu['label'] = score

    # Skills
    ## hard
    for skill in output['hardSkills']['value']:
        relevance = job.relevance([skill['value']['name'].lower()]) * skill['value']['proficiency'] / 5
        skill['label'] = relevance
        
    
    ## soft
    for skill in output['softSkills']['value']:
        relevance = job.relevance([skill['value']['name'].lower()]) * skill['value']['proficiency'] / 5
        skill['label'] = relevance
    
    # languages
    for lang in output['languages']['value']:
        relevance = job.relevance([lang['value']['name'].lower()]) * lang['value']['proficiency'] / 5
        lang['label'] = relevance
        
    # certifications
    for cert in output['certifications']['value']:
        keywords = extract_keywords(cert['value']['title'])
        relevance = job.relevance(keywords)
        cert['label'] = relevance
        
    # accomplishements
    for acc in output['accomplishments']['value']:
        keywords = extract_keywords(acc['value']['title'])
        relevance = job.relevance(keywords)
        acc['label'] = relevance
        
    # projects
    for proj in output['accomplishments']['value']:
        keywords = extract_keywords(proj['value']['title'])
        keywords.update(extract_keywords(proj['value']['description']))
        relevance = job.relevance(keywords)
        proj['label'] = relevance
        
    # extracurriculars
    for extra in output['accomplishments']['value']:
        keywords = extract_keywords(extra['value']['title'])
        keywords.update(extract_keywords(proj['value']['description']))
        relevance = job.relevance(keywords)
        extra['label'] = relevance
    
    # patents
    for patent in output['accomplishments']['value']:
        keywords = extract_keywords(patent['value']['title'])
        keywords.update(extract_keywords(proj['value']['description']))
        relevance = job.relevance(keywords)
        patent['label'] = relevance
    
    
    # interests 
    for patent in output['accomplishments']['value']:
        keywords = extract_keywords(patent['value']['title'])
        keywords.update(extract_keywords(proj['value']['description']))
        relevance = job.relevance(keywords)
        patent['label'] = relevance
    
    return output

In [96]:
out = score_resume(first, DS)
out['extracurriculars']

{'value': [{'value': {'title': '',
    'fromDate': '2010-01-23',
    'toDate': 'Today',
    'description': 'Runs a dog-shelter on the weekends for homeless dogs.'},
   'label': 0}],
 'label': 0}

In [63]:
first

{'_id': ObjectId('636b1bc1f728e7b519af0072'),
 'title': {'value': 'Data scientist', 'label': 0},
 'contactInfo': {'value': {'address': {'value': 'Stockholmsvägen 15',
    'label': 0},
   'website': {'value': '', 'label': 0},
   'linkedin': {'value': 'www.linkedin.com/in/caterina- bonazzi ', 'label': 0},
   'name': {'value': 'Caterina Bonazzi', 'label': 0},
   'phoneNumber': {'value': '', 'label': 0},
   'email': {'value': 'caterinabonazzi@gmail.com', 'label': 0},
   'github': {'value': '', 'label': 0},
   'birthday': {'value': '', 'label': 0},
   'family': {'value': 'Husband and two children', 'label': 0}},
  'label': 0},
 'summary': {'value': 'I am a passionate data scientist curious and ambitious. I love to dig into data trying to bring order in a word of chaos.',
  'label': 0},
 'experience': {'value': [{'value': {'company': 'IKEA',
     'title': 'Data Scientist',
     'location': 'Malmö, Sweden',
     'fromDate': 'September 2021',
     'toDate': 'Present',
     'description': ''},


In [49]:
from datetime import datetime


# this is just for Karl to learn, can be removed later
# for words like "today" or "present" use today()

fromDate = first['experience']['value'][1]['value']['fromDate']
print(fromDate)
datetime_object_from = datetime.strptime(fromDate, '%B %Y')
print(datetime_object_from)

toDate = first['experience']['value'][1]['value']['toDate']
print(toDate)
datetime_object_to = datetime.strptime(toDate, '%B %Y')
print(datetime_object_to)

print(datetime_object_to-datetime_object_from)

today = datetime.today()
print(today)

diff = today - datetime_object_to
days = np.floor(diff.total_seconds()/86400)
print(int(days)) # gets the number of days

November 2019
2019-11-01 00:00:00
September 2021
2021-09-01 00:00:00
670 days, 0:00:00
2022-11-09 06:24:09.355882
434


In [38]:
def get_length_of_exp(experience) -> int:
    


670 days, 0:00:00
